In [1]:
import numpy as np

def pad_text(text: str, block_size: int) -> str:
    padding_length = (block_size - len(text) % block_size) % block_size
    return text + ' ' * padding_length

def text_to_numbers(text: str) -> np.ndarray:
    return np.array([ord(char) % 256 for char in text])

def numbers_to_text(numbers: np.ndarray) -> str:
    return ''.join(chr(int(num) % 256) for num in numbers)

def mod_inverse(a: int, m: int) -> int:
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def compute_inverse_key_matrix(key_matrix: np.ndarray, mod: int = 256) -> np.ndarray:
    det = int(np.round(np.linalg.det(key_matrix)))
    det_inv = mod_inverse(det % mod, mod)

    if det_inv is None:
        raise ValueError("Key matrix is not invertible under mod 256")

    adjugate_matrix = np.round(det * np.linalg.inv(key_matrix)).astype(int) % mod
    return (det_inv * adjugate_matrix) % mod

def process_block(block: np.ndarray, key_matrix: np.ndarray, mode: str = 'encrypt') -> np.ndarray:
    if mode == 'decrypt':
        key = compute_inverse_key_matrix(key_matrix)
    else:
        key = key_matrix
    return np.dot(key, block) % 256

def hill_cipher_text(text: str, key_matrix: np.ndarray, mode: str = 'encrypt') -> str:
    n = key_matrix.shape[0]
    padded_text = pad_text(text, n)
    numbers = text_to_numbers(padded_text)
    blocks = numbers.reshape(-1, n).T
    processed_blocks = np.zeros_like(blocks)
    for i in range(blocks.shape[1]):
        processed_blocks[:, i] = process_block(blocks[:, i], key_matrix, mode)
    processed_numbers = processed_blocks.T.flatten()
    processed_text = numbers_to_text(processed_numbers)
    if mode == 'decrypt':
        processed_text = processed_text.rstrip()
    return processed_text

if __name__ == "__main__":
    key_matrix = np.array([
        [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [6, 8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [6, 8, 10, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [6, 8, 10, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [6, 8, 10, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [6, 8, 10, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [6, 8, 10, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [6, 8, 10, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [6, 8, 10, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 8, 10, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 8, 10, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 8, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 8, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    ])

    original_text = input("Enter text to send: ")
    print(f"Original text: {original_text}")

    encrypted_text = hill_cipher_text(original_text, key_matrix, mode='encrypt')
    print(f"\nEncrypted text: {encrypted_text}")

    decrypted_text = hill_cipher_text(encrypted_text, key_matrix, mode='decrypt')
    print(f"\nDecrypted text: {decrypted_text}")


Enter text to send: helooo cns
Original text: helooo cns

Encrypted text: 2???ð3>CX

Decrypted text: helooo cns
